In [1]:
import mxnet as mx
import cv2
import numpy as np
from os.path import join
import os

In [2]:
batch_size = 512
val_label = None
imgs = None

class_id = str(1)
num_id = '10'
attr_len = [int(num_id)]
eval_epoch = 30
mu = 2
width = 110 * mu
longth = int(160/8*7 * mu)
longth_ = int(longth/8*6)
width_ = int(width /8*6)
folder_name = '{}_data_{}/'.format(class_id,num_id)
class_num_id = '{}_data_{}/'.format(class_id,num_id)
resnet_version = '152'
checkpoint_name ='fine-tuned-resnet{}-{}-{}'.format(resnet_version,class_id,num_id) 
pretrain_model_name = 'resnet-{}'.format(resnet_version)




def eval_avage(fea_list,val_label):
    res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.gpu())
    val_label = val_label.as_in_context(mx.gpu())
    #res_final = final_res[0]
    for i in fea_list:
        res_final += i
    res_final/= len(fea_list)
    acc = mx.metric.Accuracy()
    top2 = mx.metric.TopKAccuracy(top_k=2)
    acc.update(labels=[val_label],preds=[res_final])
    top2.update(labels=[val_label],preds=[res_final])
    print(acc.get())
    print(top2.get())
    return res_final


In [3]:
os.chdir(join('/home/lhw/face/faceRec/',folder_name))

In [4]:
def do_multi_predict(model_name,batch_size=512,folder_name=folder_name,epoch_num=eval_epoch,tt=1):
    #sym, arg_params, aux_params = mx.model.load_checkpoint('fine-tuned-firstclass-res18-use-resize', 25)
    sym, arg_params, aux_params = mx.model.load_checkpoint(model_name,epoch_num)
    mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
    mod.bind(for_training=False, data_shapes=[('data', (batch_size,3,longth_,width_))], 
             label_shapes=mod._label_shapes)
    mod.set_params(arg_params, aux_params, allow_missing=True)
    augs = [mx.image.CenterCropAug(size=(width,longth))]
    augs += mx.image.CreateAugmenter(data_shape=(3,longth_, width_),rand_crop=True,rand_resize=False, rand_mirror=True, brightness=0.125, contrast=0.125, rand_gray=0.05,saturation=0.125, pca_noise=0, inter_method=10)
    final_res = None
    print(model_name,epoch_num)
    # define a simple data batch
    for i in range(tt):
        print(i)
        val_iter= mx.image.ImageIter(batch_size=batch_size, data_shape=(3, longth_, width_), label_width=1,
                                       path_imgidx='cloth_val.idx', path_imgrec='cloth_val.rec', shuffle=False,
                                       aug_list=augs)
        res = mod.predict(val_iter,always_output_list=True)
        if final_res is None:
            final_res = res
        else:
            final_res += res
    del mod
    return final_res
%time fea_list = do_multi_predict(checkpoint_name,tt=5,epoch_num=eval_epoch)

#res_final = final_res[0]
'''
res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.cpu())
for i in fea_list:
    res_final += i
res_final/= len(fea_list)
'''
#print(1,10,3,4,5,6,7,8)
#rrrr =  ['帽子','鞋子','披带类','上装','裤子','裙子','连体装','包包']
#print(res_final,rrrr[res_final[0].asnumpy().argmax()])

fine-tuned-resnet152-1-10 30
0
1
2
3
4
CPU times: user 1min, sys: 7.03 s, total: 1min 7s
Wall time: 48.7 s


'\nres_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.cpu())\nfor i in fea_list:\n    res_final += i\nres_final/= len(fea_list)\n'

In [5]:
print(fea_list[0].asnumpy())

[[  1.72660034e-02   6.03203021e-04   1.52707249e-02 ...,   8.85449816e-03
    5.92178218e-02   4.34029615e-03]
 [  3.54628864e-04   6.22537584e-07   8.82950280e-06 ...,   3.57272038e-05
    9.99575913e-01   9.52266703e-07]
 [  9.98102725e-01   5.24943800e-07   5.32778870e-07 ...,   5.38065251e-07
    3.32774590e-07   1.65709085e-03]
 ..., 
 [  4.13289788e-04   5.29677163e-06   8.60434193e-06 ...,   3.09992000e-03
    9.96251285e-01   7.62217096e-05]
 [  5.44314134e-05   4.07956168e-06   1.83116572e-04 ...,   2.59209238e-03
    9.96435881e-01   1.33342983e-05]
 [  4.27482352e-02   1.49002988e-02   9.21234071e-01 ...,   4.58951853e-03
    2.10917694e-03   1.89070648e-03]]


In [6]:
%time val_label= ge_val_label(folder_name=folder_name)

(1081,)
CPU times: user 11.6 s, sys: 908 ms, total: 12.5 s
Wall time: 5.85 s


In [7]:
import pickle
with open('fea_list'.format(class_num_id),'wb') as f:
    pickle.dump(fea_list,f)
with open('val_label'.format(class_num_id),'wb') as f:
    pickle.dump(val_label,f)

In [9]:
res_final= eval_avage(fea_list[0:5],val_label)
p = res_final.asnumpy()
l = val_label.asnumpy()
pred = []
ff = []
tt = []
fff = []
ttf = []
ttc = []
allf = []
the = 0.90
for i in range(len(p)):
    pp = p[i]
    mm = pp.argmax()
    if mm != l[i]:
        allf.append(pp[mm])
    if mm != l[i] and pp[mm] <the:
        ff.append(pp[mm])
    else:
        if pp[mm] < the:
            tt.append(pp[mm])
            ttc.append(mm)
print(len(tt),len(ff),len(ttf),len(fff),len(allf))
print((len(tt)+len(ff))/res_final.shape[0])
print((len(allf)-len(ff))/res_final.shape[0])
#print(len([i for i in ttc if i==])/500)

('accuracy', 0.82423681776133206)
('top_k_accuracy_2', 0.91119333950046255)
197 150 0 0 190
0.3209990749306198
0.03700277520814061


In [10]:
import pandas as pd
t = pd.read_excel('/home/lhw/face/faceRec/data.xlsx')
t = t[t.cid==int(class_id)]
t.bid = t.bid.apply(str)
t=t.sort_values(by='bid')
print(t)
class_name_list = t.bname.tolist()
print(class_name_list)
class_name_list_dic = {}
for idx,i in enumerate(class_name_list):
    class_name_list_dic[i] = idx

    cid cname  bid bname
8     1    帽子   10   雷锋帽
9     1    帽子   11   鸭舌帽
10    1    帽子  118   皮草帽
0     1    帽子    2   毛线帽
1     1    帽子    3   头巾帽
2     1    帽子    4   贝雷帽
3     1    帽子    5   报童帽
4     1    帽子    6   渔夫帽
5     1    帽子    7   太阳帽
6     1    帽子    8    礼帽
7     1    帽子    9   平顶帽
['雷锋帽', '鸭舌帽', '皮草帽', '毛线帽', '头巾帽', '贝雷帽', '报童帽', '渔夫帽', '太阳帽', '礼帽', '平顶帽']


In [11]:
preds = res_final.asnumpy()
preds = preds.argmax(axis=1)
preds.shape
from sklearn.metrics import confusion_matrix,classification_report,f1_score,recall_score,accuracy_score
cm = confusion_matrix(l,preds)

In [15]:
cm_norm = (cm/cm.sum(axis=1)[:,np.newaxis])
pprint(cm_norm)
from pprint import pprint
selected_class_type = '礼帽'
c1 = [i[class_name_list_dic[selected_class_type]] for i in cm_norm]
c2 = cm_norm[class_name_list_dic[selected_class_type]]
print(sum(c2))
pprint(sorted(list(zip(class_name_list,c1)),key=lambda s:s[1],reverse=True))
pprint(sorted(list(zip(class_name_list,c2)),key=lambda s:s[1],reverse=True))

array([[ 0.90812721,  0.        ,  0.02473498,  0.00353357,  0.01060071,
         0.02120141,  0.01060071,  0.00353357,  0.01060071,  0.00706714],
       [ 0.66666667,  0.        ,  0.33333333,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.05309735,  0.        ,  0.91150442,  0.        ,  0.        ,
         0.00884956,  0.00884956,  0.00884956,  0.00884956,  0.        ],
       [ 0.11764706,  0.        ,  0.11764706,  0.64705882,  0.        ,
         0.05882353,  0.        ,  0.        ,  0.05882353,  0.        ],
       [ 0.11764706,  0.        ,  0.01470588,  0.01470588,  0.70588235,
         0.08823529,  0.        ,  0.        ,  0.04411765,  0.01470588],
       [ 0.09230769,  0.        ,  0.03076923,  0.01538462,  0.06153846,
         0.63076923,  0.01538462,  0.04615385,  0.03076923,  0.07692308],
       [ 0.03296703,  0.        ,  0.04395604,  0.01098901,  0.03296703,
         0.04395604,  0.64835165,  0.08791209

In [14]:
pprint(classification_report(y_pred=preds,y_true=l,target_names=class_name_list))


('             precision    recall  f1-score   support\n'
 '\n'
 '        雷锋帽       0.87      0.91      0.89       283\n'
 '        鸭舌帽       0.00      0.00      0.00         3\n'
 '        皮草帽       0.82      0.91      0.87       113\n'
 '        毛线帽       0.69      0.65      0.67        17\n'
 '        头巾帽       0.73      0.71      0.72        68\n'
 '        贝雷帽       0.65      0.63      0.64        65\n'
 '        报童帽       0.76      0.65      0.70        91\n'
 '        渔夫帽       0.85      0.85      0.85       178\n'
 '        太阳帽       0.88      0.86      0.87       250\n'
 '         礼帽       0.38      0.46      0.41        13\n'
 '\n'
 'avg / total       0.82      0.82      0.82      1081\n')


/home/lhw/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 10, does not match size of target_names, 11
  .format(len(labels), len(target_names))
/home/lhw/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
